In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import cgal_Segmentation_Module as csm
#help(csm)


# In[ ]:


#####2 goals
"""1)  create the final datajoint tables to store the final spine meshs in:
    a. One table to store the segment data (linked to the components table)
    b. One table to store the labeled mesh vertices and faces of the neurons (linked to CleanseMesh)
2) Create the function that goes through and writes the CGAL library for all of the components
"""


# In[ ]:


"""Pseduo code for function that goes through and writes the CGAL library for all of the components

1) Recieve list of neurons to do and a flag that when set will look to
    another table for the clustering parameter for each neuron
2) Pull down the neurons mesh data from cleansedMesh
3) Pull down all the components with the neuron ID that have size > 100
4) For each component:
5) Generate the off file:
    ---------------Way I do it in the blender file----------------
    In load_Neuron_automatic_spine, download whole mesh
    a. Before create the mesh object send faces and verts to filter_verts_and_faces
    b. filter_verts_and_faces:
        downloads the indexes for the compartment
        Only saves off the verts that are mentioned in the indexes
        Only saves off the faces that are mentioned in the indexes
            returns them
    c. builds the off file by:
        Finding the faces that have all indices included in the verts list
        finish with the write_Part_Neuron_Off_file
    -------------------
    1. create the file name string: "neuron_" + str(segment_id) + "_" + str(compartment_type_name) + "_" + str(found_component_index)
    2. get the number of indices and faces
    3. Open them and write them to the file
    4. For the vertices:
        For each index in the vertices blob of the components table, 
         write the coordinates in the index location of the Cleansed mesh table
            while creating a lookup dictionary where it has old_vert_index:new_index  (vert_lookup)
    5. For the faces:
        For each index in the faces blob of the components table,
             Get the new index by (vert_lookup) and save to list
        Write the list to the file
    
    Call the CGAL function to generate the labels
    String calculat the CGAL file name and the CGAL SDF value 
    Write the two lists to the datajoint table (linked to components)
    """




# In[ ]:


import datajoint as dj
import numpy as np
import datetime
import math

#from cloudvolume import CloudVolume
#from collections import Counter
#from funconnect import ta3


# In[ ]:


#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20


# user: celiib
# pass: newceliipass
# host: at-database.ad.bcm.edu
# schemas: microns_% and celiib_%


# In[ ]:


schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


# In[ ]:


#if temp folder doesn't exist then create it
import os
if (os.path.isdir(os.getcwd() + "/temp")) == False:
    os.mkdir("temp")


# In[ ]:


import os
import pathlib

def generate_component_off_file(neuron_ID, compartment_type, component_id, n_vertex_indices, n_triangle_indices, 
                                vertex_indices, triangle_indices,vertices, triangles):
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / "temp"
    filename = "neuron_" + str(neuron_ID) + "_" + str(compartment_type) + "_" + str(component_id)
    path_and_filename = file_loc / filename
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(n_vertex_indices) + " " + str(n_triangle_indices) + " 0\n" )
    
    #start writing all of the vertices
    """
        4. For the vertices:
        For each index in the vertices blob of the components table, 
         write the coordinates in the index location of the Cleansed mesh table
            while creating a lookup dictionary where it has old_vert_index:new_index  (vert_lookup)
    """       
    verts_lookup = {}
    for i, vin in enumerate(vertex_indices):
        #get the coordinates of the vertex
        coordinates = vertices[vin]
        #write the coordinates to the off file
        f.write(str(coordinates[0]) + " " + str(coordinates[1]) + " " + str(coordinates[2])+"\n")
        #create lookup dictionary for vertices
        verts_lookup[vin] = i
    
    """    5. For the faces:
        For each index in the faces blob of the components table,
             Get the new index by (vert_lookup) and save to list
        Write the list to the file"""
    for i,fac in enumerate(triangle_indices):
        verts_in_fac = triangles[fac]
        #write the verties to the off file
        f.write("3 " + str(verts_lookup[verts_in_fac[0]]) + " " + str(verts_lookup[verts_in_fac[1]]) + " " + str(verts_lookup[verts_in_fac[2]])+"\n")
        
    
    print("Done making OFF file " + str(filename))
    #return the name of the off file you created and the location
    return str(path_and_filename),str(filename)


# In[ ]:


#################THE ONE WE ARE USING
import cgal_Segmentation_Module as csm
import csv
import decimal
import time
import os

@schema
class ComponentAutoSegmentOrphan(dj.Computed):
    definition = """
    # creates the labels for the mesh table
    -> ta3p100.CompartmentOrphan.ComponentOrphan
    clusters     : tinyint unsigned  #what the clustering parameter was set to
    smoothness   : decimal(3,2)             #what the smoothness parameter was set to, number betwee 0 and 1
    ---
    n_triangles  : int unsigned # number of faces
    seg_group    : longblob     # group segmentation ID's for faces from automatic CGAL segmentation
    sdf          : longblob     #  width values for faces from from automatic CGAL segmentation
    median_sdf   : decimal(6,5) # the median width value for the sdf values
    mean_sdf     : decimal(6,5) #the mean width value for the sdf values
    third_q      : decimal(6,5) #the upper quartile for the mean width values
    ninety_perc  : decimal(6,5) #the 90th percentile for the mean width values
    time_updated : timestamp    # the time at which the segmentation was performed
   """
    
    key_source = ta3p100.CompartmentOrphan.ComponentOrphan & 'n_triangle_indices>100' & [dict(compartment_type=comp) for comp in ['Basal', 'Apical', 'Oblique', 'Dendrite']]
    
    whole_neuron_dicts = dict()
    
    def make(self, key):
        print("key = " + str(key))
        #key passed to function is just dictionary with the following attributes
        """segmentation
        segment_id
        decimation_ratio
        compartment_type
        component_index
        """
        start_time = time.time()
        
        #clusters_default = 18
        smoothness = 0.04

        Apical_Basal_Oblique_default = [12]
        basal_big = [16]

        neuron_ID = key["segment_id"]
        component = (ta3p100.CompartmentOrphan.ComponentOrphan & key).fetch1()        

        component_id = component["component_index"]
        compartment_type = component["compartment_type"]
        component_size = int(component["n_triangle_indices"])

        print("component_size = " + str(component_size))

        if (compartment_type == "Basal") & (component_size > 160000):
            cluster_list = basal_big
        else:
            cluster_list = Apical_Basal_Oblique_default


        for clusters in cluster_list:
            smoothness = 0.04
            print(str(component["segment_id"]) + " type:" + str(component["compartment_type"]) 
                      + " index:" + str(component["component_index"]) + " cluster:" + str(clusters) 
                  + " smoothness:" + str(smoothness))

            #generate the off file for each component
            #what need to send them:
            """----From cleansed Mesh---
            vertices
            triangles
            ----From component table--
            n_vertex_indices
            n_triangle_indices
            vertex_indices
            triangle_indices"""
            
            if key['segment_id'] not in self.whole_neuron_dicts:
                self.whole_neuron_dicts[key['segment_id']] = (ta3p100.CleansedMeshOrphan & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
            
            path_and_filename, off_file_name = generate_component_off_file(neuron_ID, compartment_type, component_id,
                                        component["n_vertex_indices"],
                                        component["n_triangle_indices"],
                                        component["vertex_indices"],
                                        component["triangle_indices"],
                                        self.whole_neuron_dicts[key['segment_id']]["vertices"],
                                        self.whole_neuron_dicts[key['segment_id']]["triangles"])
            
            print(len(component['vertex_indices']), len(component['triangle_indices']))
            
            #will have generated the component file by now so now need to run the segmentation

            csm.cgal_segmentation(path_and_filename,clusters,smoothness)

            #generate the name of the files
            cgal_file_name = path_and_filename + "-cgal_" + str(clusters) + "_"+str(smoothness)
            group_csv_cgal_file = cgal_file_name + ".csv"
            sdf_csv_file_name = cgal_file_name+"_sdf.csv"

            
            try:
                with open(group_csv_cgal_file) as f:
                  reader = csv.reader(f)
                  your_list = list(reader)
                group_list = []
                for item in your_list:
                    group_list.append(int(item[0]))

                with open(sdf_csv_file_name) as f:
                  reader = csv.reader(f)
                  your_list = list(reader)
                sdf_list = []
                for item in your_list:
                    sdf_list.append(float(item[0]))
            except:
                print("no CGAL segmentation for " + str(off_file_name) )
                return

            #print(group_list)
            #print(sdf_list)

            #now write them to the datajoint table  
            #table columns for ComponentAutoSegmentation: segmentation, segment_id, decimation_ratio, compartment_type, component_index, seg_group, sdf
#             print(dict(key,
#                                 clusters=clusters,
#                                 smoothness=smoothness,
#                                 n_triangles=component["n_triangle_indices"],
#                                 seg_group=group_list,
#                                 sdf=sdf_list,
#                                 median_sdf=np.median(sdf_list),
#                                 mean_sdf=np.mean(sdf_list),
#                                 third_q=np.percentile(sdf_list, 75),
#                                 ninety_perc=np.percentile(sdf_list, 90),
#                                 time_updated=str(datetime.datetime.now())[0:19]))
            
            comp_dict = dict(key,
                                clusters=clusters,
                                smoothness=smoothness,
                                n_triangles=component["n_triangle_indices"],
                                seg_group=group_list,
                                sdf=sdf_list,
                                median_sdf=np.median(sdf_list),
                                mean_sdf=np.mean(sdf_list),
                                third_q=np.percentile(sdf_list, 75),
                                ninety_perc=np.percentile(sdf_list, 90),
                                time_updated=str(datetime.datetime.now())[0:19])

            self.insert1(comp_dict)

            #then go and erase all of the files used: the sdf files, 
            real_off_file_name = path_and_filename + ".off"

            files_to_delete = [group_csv_cgal_file,sdf_csv_file_name,real_off_file_name]
            for fl in files_to_delete:
                if os.path.exists(fl):
                    os.remove(fl)
                else:
                    print(fl + " file does not exist")

        print("finished")
        print("--- %s seconds ---" % (time.time() - start_time))


# In[ ]:


ComponentAutoSegmentOrphan.populate(reserve_jobs=True)
print("finished")



Connecting celiib@10.28.0.34:3306
key = {'segmentation': 2, 'segment_id': 648518346341352006, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Dendrite', 'component_index': 0}
component_size = 50802
648518346341352006 type:Dendrite index:0 cluster:12 smoothness:0.04
Done making OFF file neuron_648518346341352006_Dendrite_0
25449 50802
finished
--- 13.841903686523438 seconds ---
key = {'segmentation': 2, 'segment_id': 648518346341352223, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Dendrite', 'component_index': 0}
component_size = 77823
648518346341352223 type:Dendrite index:0 cluster:12 smoothness:0.04
Done making OFF file neuron_648518346341352223_Dendrite_0
39026 77823
finished
--- 20.48962092399597 seconds ---
key = {'segmentation': 2, 'segment_id': 648518346341353019, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Dendrite', 'component_index': 0}
component_size = 17592
648518346341353019 type:Dendrite index:0 cluster:12 smoothness:0.04
Done making O

In [1]:

import datajoint as dj
import numpy as np
import datetime
import math

#from cloudvolume import CloudVolume
#from collections import Counter
#from funconnect import ta3


# In[ ]:


#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20


# user: celiib
# pass: newceliipass
# host: at-database.ad.bcm.edu
# schemas: microns_% and celiib_%


# In[ ]:


schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [13]:
ta3p100.ComponentAutoSegmentOrphan()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,clusters what the clustering parameter was set to,"smoothness what the smoothness parameter was set to, number betwee 0 and 1",n_triangles number of faces,seg_group group segmentation ID's for faces from automatic CGAL segmentation,sdf width values for faces from from automatic CGAL segmentation,median_sdf the median width value for the sdf values,mean_sdf the mean width value for the sdf values,third_q the upper quartile for the mean width values,ninety_perc the 90th percentile for the mean width values,time_updated the time at which the segmentation was performed
2,648518346341352006,0.35,Dendrite,0,12,0.04,50802,=BLOB=,=BLOB=,0.31245,0.34377,0.51871,0.62923,2019-02-13 06:18:55
2,648518346341352223,0.35,Dendrite,0,12,0.04,77823,=BLOB=,=BLOB=,0.42073,0.39614,0.59719,0.68472,2019-02-13 06:19:15
2,648518346341353019,0.35,Dendrite,0,12,0.04,17592,=BLOB=,=BLOB=,0.42656,0.42708,0.62205,0.72423,2019-02-13 06:19:20
2,648518346341353186,0.35,Dendrite,0,12,0.04,86889,=BLOB=,=BLOB=,0.38283,0.40735,0.64874,0.72495,2019-02-13 06:19:43
2,648518346341353574,0.35,Basal,0,12,0.04,1742,=BLOB=,=BLOB=,0.68498,0.62487,0.75911,0.83389,2019-02-13 06:19:44
2,648518346341353574,0.35,Basal,1,12,0.04,90304,=BLOB=,=BLOB=,0.24122,0.27205,0.36055,0.51533,2019-02-13 06:20:11
2,648518346341353607,0.35,Dendrite,0,12,0.04,136186,=BLOB=,=BLOB=,0.53975,0.44625,0.70899,0.78397,2019-02-13 06:20:59
2,648518346341353788,0.35,Dendrite,0,12,0.04,19506,=BLOB=,=BLOB=,0.50977,0.44528,0.75302,0.81860,2019-02-13 06:21:05
2,648518346341353883,0.35,Dendrite,0,12,0.04,77210,=BLOB=,=BLOB=,0.26132,0.39540,0.68802,0.78412,2019-02-13 06:21:31
2,648518346341354048,0.35,Dendrite,0,12,0.04,6435,=BLOB=,=BLOB=,0.28300,0.36271,0.59676,0.75505,2019-02-13 06:21:33


In [5]:
key_source = ta3p100.CompartmentOrphan.ComponentOrphan & 'n_triangle_indices>100' & [dict(compartment_type=comp) for comp in ['Basal', 'Apical', 'Oblique', 'Dendrite']]

In [6]:
key_source

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,n_vertex_indices,n_triangle_indices,vertex_indices preserved indices of each vertex of this sub-compartment,triangle_indices preserved indices of each triangle of this sub-compartment
2,648518346341352006,0.35,Dendrite,0,25449,50802,=BLOB=,=BLOB=
2,648518346341352223,0.35,Dendrite,0,39026,77823,=BLOB=,=BLOB=
2,648518346341353019,0.35,Dendrite,0,8805,17592,=BLOB=,=BLOB=
2,648518346341353186,0.35,Dendrite,0,43630,86889,=BLOB=,=BLOB=
2,648518346341353574,0.35,Basal,0,891,1742,=BLOB=,=BLOB=
2,648518346341353574,0.35,Basal,1,45239,90304,=BLOB=,=BLOB=
2,648518346341353607,0.35,Dendrite,0,68234,136186,=BLOB=,=BLOB=
2,648518346341353788,0.35,Dendrite,0,9789,19506,=BLOB=,=BLOB=
2,648518346341353883,0.35,Dendrite,0,38697,77210,=BLOB=,=BLOB=
2,648518346341354048,0.35,Dendrite,0,3245,6435,=BLOB=,=BLOB=


In [22]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20


# user: celiib
# pass: newceliipass
# host: at-database.ad.bcm.edu
# schemas: microns_% and celiib_%


# In[ ]:


schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')

ta3p100.ComponentLabelOrphan()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,clusters what the clustering parameter was set to,"smoothness what the smoothness parameter was set to, number betwee 0 and 1",time_updated the time at which the component labels were updated,n_vertices number of vertices in component,n_triangles number of faces in component,"labeled_vertices indicate which vertices are spine,spine_head,spine_neck otherwise 0","labeled_triangles indicate which faces are spine,spine_head,spine_neck otherwise 0","n_heads totals the number of heads after classification, helps for optimization","used_version whether this component is used in the final labels or not, 0 no, 1 yes"
2,648518346341352006,0.35,Dendrite,0,12,0.04,2019-02-13 13:32:27,25449,50802,=BLOB=,=BLOB=,127,1
2,648518346341352223,0.35,Dendrite,0,12,0.04,2019-02-13 13:32:49,39026,77823,=BLOB=,=BLOB=,173,1
2,648518346341353019,0.35,Dendrite,0,12,0.04,2019-02-13 13:32:04,8805,17592,=BLOB=,=BLOB=,57,1
2,648518346341353186,0.35,Dendrite,0,12,0.04,2019-02-13 13:32:56,43630,86889,=BLOB=,=BLOB=,166,1
2,648518346341353574,0.35,Basal,0,12,0.04,2019-02-13 13:32:03,891,1742,=BLOB=,=BLOB=,0,1
2,648518346341353574,0.35,Basal,1,12,0.04,2019-02-13 13:32:55,45239,90304,=BLOB=,=BLOB=,154,1
2,648518346341353607,0.35,Dendrite,0,12,0.04,2019-02-13 13:33:16,68234,136186,=BLOB=,=BLOB=,184,1
2,648518346341353788,0.35,Dendrite,0,12,0.04,2019-02-13 13:32:03,9789,19506,=BLOB=,=BLOB=,23,1
2,648518346341353883,0.35,Dendrite,0,12,0.04,2019-02-13 13:32:46,38697,77210,=BLOB=,=BLOB=,167,1
2,648518346341354048,0.35,Dendrite,0,12,0.04,2019-02-13 13:32:02,3245,6435,=BLOB=,=BLOB=,12,1


In [23]:
schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decimation,3f6d204420095d24e5871213fe8fbf1a,reserved,=BLOB=,,=BLOB=,jcotton@10.28.3.179,at-compute004,188200,1958721,2019-01-10 21:49:37
__decimation,7b1feda329b85293476bee7d0acf42fb,reserved,=BLOB=,,=BLOB=,jcotton@10.28.3.179,at-compute004,186672,1958694,2019-01-10 21:37:01
